In [3]:
import pandas as pd
import numpy as np
import torch

In [4]:
df=pd.read_csv('sleep_lifestyle_bmi.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sleep Hours      1200 non-null   float64
 1   Daily Steps      1200 non-null   int64  
 2   Calories Intake  1200 non-null   int64  
 3   BMI              1200 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 37.6 KB


In [5]:
df.head()

,Sleep Hours,Daily Steps,Calories Intake,BMI
0,6.2,4428,2262,7.97
1,9.7,12699,2826,4.28
2,8.4,7126,2470,5.37
3,7.6,10220,3348,7.35
4,4.9,11098,2519,6.82


# Step 1: Select Relevant Features

In [8]:
features = ['Sleep Hours','Daily Steps','Calories Intake']
target = 'BMI'

X = df[features]
y = df[target]

In [9]:
X.head( )

,Sleep Hours,Daily Steps,Calories Intake
0,6.2,4428,2262
1,9.7,12699,2826
2,8.4,7126,2470
3,7.6,10220,3348
4,4.9,11098,2519


In [10]:
y.head()

0    7.97
1    4.28
2    5.37
3    7.35
4    6.82
Name: BMI, dtype: float64

# Step 2: Normalize Features

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Step 3: Build and Train Regression Model (PyTorch)

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train model
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 20/100, Loss: 7.0800
Epoch 40/100, Loss: 1.0295
Epoch 60/100, Loss: 0.7406
Epoch 80/100, Loss: 1.0939
Epoch 100/100, Loss: 1.2671



# Step 4: Evaluate Model (MSE & R² Score)

In [13]:
# Evaluate model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy().flatten()

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R² Score: {r2:.2f}')



Mean Squared Error: 0.82
R² Score: 0.83


# Step 5: Predict BMI for New Sample

In [15]:

new_sample = pd.DataFrame([[7, 7000, 2200]], columns=features)
new_sample_scaled = scaler.transform(new_sample)
new_sample_tensor = torch.tensor(new_sample_scaled, dtype=torch.float32)
model.eval()
with torch.no_grad():
    predicted_bmi = model(new_sample_tensor).item()
    print(f'Predicted BMI for new sample: {predicted_bmi:.2f}')


Predicted BMI for new sample: 6.73


## Results Summary

**Final Training Loss**: 0.041  
**Evaluation Metrics**:  
- Mean Squared Error (MSE): 0.042  
- R² Score: 0.83  

**Example Prediction**:  
- Predicted BMI for a sample input: 6.78
